# day 8

https://adventofcode.com/8/day/8

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day08.txt')

LOGGER = logging.getLogger('day08')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)"""

test_data_2 = """LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

#### function def

In [ ]:
def parse_data(s: str):
    inst, netstr = s.split('\n\n')
    inst = list(inst.strip())
    net = {}
    for line in netstr.split('\n'):
        node, neighbor_str = line.split(" = ")
        l_node, r_node = neighbor_str.replace('(', '').replace(')', '').split(', ')
        net[node] = {"L": l_node, "R": r_node}
    return inst, net

In [ ]:
def q_1(data):
    instructions, net = parse_data(data)
    steps = 0
    node = 'AAA'
    i_inst = 0
    L = len(instructions)
    while node != 'ZZZ':
        inst = instructions[i_inst % L]
        node = net[node][inst]
        steps += 1
        i_inst += 1
    return steps

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 2
    assert q_1(test_data_2) == 6
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

In [ ]:
test_data = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)"""

#### function def

In [ ]:
def find_starting_nodes(net):
    return sorted(n for n in net if n[-1] == 'A')

In [ ]:
instructions, net = parse_data(test_data)
assert find_starting_nodes(net) == ['11A', '22A']

In [ ]:
instructions, net = parse_data(load_data())
assert find_starting_nodes(net) == ['AAA', 'MHA', 'NBA', 'TTA', 'VVA', 'XSA']

In [ ]:
def find_cycles(instructions, net, starting_node):
    """starting at one node, walk the instructions until we have revisited one of the Z
    nodes *at the same instruction position*. along the way, record the number of steps
    at which that occurred. the first time we re-visit a Z node we have a cycle.
    Once every key in the cycle dictionary has at least 2 values, we have a complete
    cycle list and can turn that into a list of steps at which we will be at a Z node

    """
    i_inst = steps = 0
    L = len(instructions)
    node = starting_node
    z_cycles = {}
    while True:
        inst = instructions[i_inst]
        node = net[node][inst]
        steps += 1
        if node[-1] == 'Z':
            k = node, i_inst
            if k not in z_cycles:
                z_cycles[k] = []

            z_cycles[k].append(steps)
        if len(z_cycles) > 0 and all(len(v) >= 2 for v in z_cycles.values()):
            return z_cycles
        i_inst += 1
        i_inst %= L

In [ ]:
instructions, net = parse_data(test_data)
assert find_cycles(instructions, net, '11A') == {('11Z', 1): [2, 4]}
assert find_cycles(instructions, net, '22A') == {('22Z', 0): [3, 9], ('22Z', 1): [6, 12]}

In [ ]:
def cycle_to_list(cycle, n_steps):
    step_0, step_1 = cycle
    delta = step_1 - step_0
    return [step_0 + delta * i for i in range(0, n_steps + 1)]

In [ ]:
def cycle_set_to_list(cycle_set, n_steps):
    o = {v
         for cycle in cycle_set
         for v in cycle_to_list(cycle=cycle, n_steps=n_steps)}
    return sorted(o)

In [ ]:
assert cycle_to_list([2, 4], 4) == [2, 4, 6, 8, 10]
assert cycle_to_list([3, 9], 5) == [3, 9, 15, 21, 27, 33]
assert cycle_to_list([6, 12], 4) == [6, 12, 18, 24, 30]
assert cycle_set_to_list([[3, 9], [6, 12]], 5) == [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36]

In [ ]:
def find_shared_values(step_lists):
    shared = None
    for step_list in step_lists:
        if shared is None:
            shared = set(step_list)
        else:
            shared = shared.intersection(step_list)
    return sorted(shared)

assert find_shared_values([[1, 2, 3], [3, 6, 9]]) == [3]

In [ ]:
# these functions from https://math.stackexchange.com/a/3864593

def extended_gcd(a, b):
    """Extended Greatest Common Divisor Algorithm

    Returns:
        gcd: The greatest common divisor of a and b.
        s, t: Coefficients such that s*a + t*b = gcd

    Reference:
        https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Pseudocode
    """
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r:
        quotient, remainder = divmod(old_r, r)
        old_r, r = r, remainder
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    return old_r, old_s, old_t


def combine_phased_rotations(a_period, a_phase, b_period, b_phase):
    """Combine two phased rotations into a single phased rotation

    Returns: combined_period, combined_phase

    The combined rotation is at its reference point if and only if both a and b
    are at their reference points.
    """
    gcd, s, t = extended_gcd(a_period, b_period)
    phase_difference = a_phase - b_phase
    pd_mult, pd_remainder = divmod(phase_difference, gcd)
    if pd_remainder:
        raise ValueError("Rotation reference points never synchronize.")

    combined_period = a_period // gcd * b_period
    combined_phase = (a_phase - s * pd_mult * a_period) % combined_period
    return combined_period, combined_phase

In [ ]:
def cycles_to_phased_rotations(cycles):
    return {starting_node: [[cycle[1] - cycle[0], cycle[1]]
                            for (ending_node, cycle) in cycle_dict.items()]
            for (starting_node, cycle_dict) in cycles.items()}

instructions, net = parse_data(test_data)
starting_nodes = find_starting_nodes(net)
cycles = {starting_node: find_cycles(instructions=instructions, net=net, starting_node=starting_node)
          for starting_node in starting_nodes}

assert cycles_to_phased_rotations(cycles) == {'11A': [[2, 4]], '22A': [[6, 9], [6, 12]]}

In [ ]:
def combine_cycles(cycles):
    phased_rotations = cycles_to_phased_rotations(cycles=cycles)
    combined = [[0, 1]]
    for (starting_node, phased_rotation_list) in phased_rotations.items():
        combined = [list(reversed(combine_phased_rotations(a_period=current_pr[1], a_phase=current_pr[0],
                                                           b_period=new_pr[1], b_phase=new_pr[0])))
                    for current_pr in combined
                    for new_pr in phased_rotation_list]
    return combined

instructions, net = parse_data(test_data)
starting_nodes = find_starting_nodes(net)
cycles = {starting_node: find_cycles(instructions=instructions, net=net, starting_node=starting_node)
          for starting_node in starting_nodes}
assert combine_cycles(cycles) == [[6, 36], [6, 12]]

In [ ]:
def q_2(data):
    instructions, net = parse_data(data)
    starting_nodes = find_starting_nodes(net)
    cycles = {starting_node: find_cycles(instructions=instructions, net=net, starting_node=starting_node)
              for starting_node in starting_nodes}
    combined_cycles = combine_cycles(cycles)
    return min(a for (a, b) in combined_cycles)

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 6
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin